<a href="https://colab.research.google.com/github/bhanu0925/NLP/blob/main/CRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import os
ROOT = "/content/drive/MyDrive/NLP/CRM"
os.chdir(ROOT)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install wordsegment

In [117]:
import pandas as pd
import pickle
import nltk
import re
import string
from wordsegment import load, segment
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from textblob import TextBlob


# Data Cleaning

In [5]:
leads = pd.read_excel("1000_leads.xlsx")

In [ ]:
leads.head(100)

In [ ]:
leads['Status information'][750]

'23/MaY/moHan:attnde demo 23/MaY/moHan:he want in june today check and will make token payment 30/MaY/moHan:he will research and do token payment 7/JunE/moHan:he will check tmng nd respnd in 5dys 13/JunE/moHan:buay with work some othr time 13/JulY/moHan: call me mndy 2|AuG|moHan:i will try'

Data normalization

1. remove data and name of the user,11/8/prema:, 10/8/17(Surendra):, 9|aUg|moHan:,31/5/17(gowtham)
2. spelling corrections
3. expand short forms like, n/w,banglr,discus,othr,intrstd,mrg,cal,tmrw, cert, n, reg, 1hr, 23rd , @5pm , ofc, 6:30 AM, amt,hav,wil,b e,'ssbb', 'ssgb', etc
4. constructions : i'm, don't,
5. RNR = ringing no response,CT = classroom training
6. Spelling corrections
7. Remove empty rows
8. replace numbers with words


In [ ]:
#!pip install clean-text

In [ ]:
leads.info()

In [6]:
leads.dropna(axis=0,inplace=True)

In [ ]:
leads.info()

In [7]:
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

In [8]:
def data_cleaning(text):

  ## lowercase
  text = lower_casing_text(text)
  ## remove puctuation
  text = remove_puctuation(text)
  ## remove digits with trailing and preceeding text - sdf243abvc, 9am
  text = remove_digits_with_words(text)
  ## Normalize words

  ## word segmentations - > shareddetails -> shared details

  ## 
  return text


In [9]:
def remove_puctuation(text):
  """
    The function will replace punctuation with a space.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text with no punctuations
         
    Example:
    Input : The World is Full of Pan%tu#at!io#n@s!
    Output : the world is full of Pantuations
    
    """
  text = text.replace(':',' ')  
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  return text

In [10]:
def remove_digits_with_words(text):
  
  """
  The function will remove digits with traiing or preceedign text.

  arguments:
        input_text: "text" of type "String".
        
  return:
        value: text with no digits with texts
        
  Example:
  Input : The World is Full of num234num, 9am
  Output : the world is full of 

  """

  text = re.sub('\w*\d\w*', '', text) #digits with trailing or preceeding text
  return text

In [115]:
SHORTHAND_MAP = {    

"abt"           : "about",
"addres"        : "address",
"adress"        : "address",
"adrs"          : "address",
"afterwardds"   : "afterwords",
"aftr"          : "after",
"aftrnun"       : "afternoon",
"alrdy"         : "already",
"alredy"        : "already",
"alrady"        : "already",
"altdy"         : "alternet day",
"amt"           : "amount",
"anothr"        : "another",
"anthr"         : "another",
"appt"          : "appointment",
"area"          : "are",
"askng"         : "asking",
"ateend"        : "attend",
"atend"         : "attend",
"atended"       : "attended",
"atnd"          : "attend",
"atnded"        : "attended",
"atteded"       : "attended",
"attedn"        : "attend",
"attednded"     : "attended",
"attendind"     : "attending",
"attende"       : "attend",
"attnd"         : "attend",
"attnde"        : "attend",
"attnded"       : "attended",
"attndng"       : "attending",
"attned"        : "attend",
"attnede"       : "attend",
"attneded"      : "attended",
"attrended"     : "attended",
"atttend"       : "attend",
"aug"           : "august",
"auugust"       : "august",
"ad"            : "and",
"availbe"       : "available",
"avalable"      : "available",

"banglr"        : "bangalore",
"blr"           : "bangalore",
'blw'           : "bangalore",
"bangaluru"     : "bangalore",
"banglore"      : "bangalore",
"mg"            : "mahathma gandhi",
"hosp"          : "hospital",
"bnglr"         : "bangalore",
"bnlgr"         :"bangalore",
"bacth"         : "batch",
"bsy"           : "busy",
"btwn"          : "between",
"b"             : "be",

"cant"          : "can not",
"cert"          : "certificate",
"cehck"         : "check",
"checkd"        : "checked",
"chck"          : "check",
"chek"          : "chek",
"chekc"         : "check",
"cal"           : "call",
"cald"          : "called",
"cacl"          : "call",
"claaroom"      : "classroom",
"clas"          : "class",
"cls"           : "classroom",
"clsroom"       : "classroom",
"CT"            : "classroom training",
"clsses"        : "classes",
"clsdemo"       : "class demo",
"clsrom"        : "classroom",
"come"        : "come",
"cmng"        : "coming",
"cmpny"       : "company",
"cncting"     : "connecting",
"cnectng"     : "connecting",
"cnnctng"     : "connecting",
"cnnecting"   : "connecting",
"cntcd"       : "contacted",
"cnfrm"       : "confirm",
"cntry"       : "contry",
"curse"       : "course",
"congirm"     : "confirm",

"dat"         : "that",
"dbs"         : "database",
"dec"         : "december",
"deffintly"   : "definitely",
"demp"        : "demo",
"dmo"         : "demo",
"den"         : "then",
"devps"       : "devops",
"dfrnt"       : "different",
"diconecting" : "disconnecting",
"diconnected" : "disconnected",
"didnt"       : "did not",
"dint"        : "did not",
"dis"         : "this",
"dng"         : "doing",
"dnt"         : "do not",
"doctors"     : "docker",
"donno"       : "do not know",
"drope"       : "drop",
"drpd"        : "dropped",
"drvg"        : "driving",
"dsmldl"      : "datascience machine learning deep learning",
"ds"          : "datascience",
"dsnt"        : "does not",
"dys"         : "days",
"dont"        : "do not",
"discus"      : "discuss",


"eachble"     : "reachable",
"eclr"        : "clear",
"emis"        : "mothly installments",
"eng"         : "engage",
"engd"        : "engaged",
"enrld"       : "enrolled",
"enrl"        : "enroll",
"even"        : "evening",
"eveng"       : "evening",
"evng"        : "evening",
"evg"         : "evening",
"exp"         : "experience",
"exler"       : "excelr",
"exlr"        : "excelr",

"fald"        : "failed",
"falg"        : "following",
"finalise"    : "finalize",
"finl"        : "final",
"finlised"    : "finalized",
"finlse"      : "finalize",
"finalised"   : "finalized",
"finlsed"     : "finalized",
"fr"          : "for",
"frnd"        : "friend",
"frnds"       : "friends",
"frshr"       : "fresher",

"gng"         : "going",
"gud"         : "good",

"hav"         : "have",
"havent"      : "have not",
"heath"       : "health",
"hom"         : "home",
"hr"          : "hour",
"hrs"         : "hours",
"hv"          : "have",
"hvng"        : "having",
"hyd"         : "hyderabad",
"hw"          : "how",


"if"          : "if",
"il"          : "i will",
"im"          : "i am",
"iam"         : "i am",
"intro"       : "introduction",
"intrrstd"    : "interested",
"intrsd"      : "interested",
"intrstd"     : "interested",
"intrtstd"    : "interested",
"inttstd"     : "interested",
"itrstd"      : "interested",
"ion"         : "ios",
"inquired"    : "enquired",

"joind"       : "joined",
"joing"       : "joining",
"jul"         : "july",
"jun"         : "june",


"lkng"        : "looking",
"lokng"       : "looking",
"latr"        : "later",

"oopng"       : "looping",

"mai"         : "mail",
"mar"         : "march",
"marow"       : "tommorow",
"marrow"      : "tommorow",
"midum"       : "medium",
"mints"       : "minutes",
"minz"        : "minutes",
"mis"         : "miss",
"misd"        : "missed",
"misg"        : "missing",
"ml"          : "machine learning",
"mndy"        : "monday",
"mndytusdy"   : "monday tuesday",
"mor"         : "more",
"mrg"         : "morning",
"mrng"        : "morning",
"mndy"        : "monday",
"marag"       : "marriage",


"ned"         : "need",
"nov"         : "november",
"nt"          : "not",
"nw"          : "network",
"nyt"         : "night",
"nxt"         : "next",
"n"           : 'and',
"nd"          : "and",

"oct"         : "october",
"off"         : "office",
"offf"        : "office",
"offrd"       : "offered",
"ofc"         : "office",
"othr"        : "other",

"paln"        : "plan",
"paning"      : "planning",
"persnl"      : "personal",
"persnly"     : "personaly",
"phn"         : "phone",
"planing"     : "planning",
"plcd"        : "placed",
"plng"        : "planning",
"plz"         : "please",
"plaz"        : "please",
"plzt"        : "please",
"pm"          : "please message",
"pone"        : "phone",
"postpnd"     : "postponed",
"prblm"       : "problem",
"prev"        : "previous",
"prjt"        : "project",
"prjts"       : "project",
"provd"       : "provide",
"prvdng"      : "providing",
"psbl"        : "possible",
"pwr"         : "power",


"rt"          : "right",
"rchble"      : "reachable",
"rd"          : "road",
"r"           : "are",
"reable"      : "reachable",
"reacahble"   : "reachable",
"reachble"    : "reachable",
"reachlbe"    : "reachable",
"reahble"     : "reachable",
"rechble"     : "reachable",
"rechd"       : "reached",
"reg"         : "register",
"repnd"       : "respond",
"respd"       : "respond",
"respnd"      : "respond",
"rnrr"        : "ringing no response",
"rnr"         : "ringing no response",
"rom"         : "room",
'rprog'       : "rprogramming",

"satdy"       : "saturday",
"sat"         : "saturday",
"shred"       : "shared",
"shft"        : "shift",
"sasr"        : "sas", 
"schdle"      : "schedule",
"scheduke"    : "schedule",
"sep"         : "september",
"sept"        : "september",
"sesion"      : "session",
"shrd"        : "shared",
"shred"       : "shared",
"shw"         : "show",
"sis"         : "sister",
"sistr"       : "sister",
"soma"        : "some",
"spcl"        : "special",
"sprk"        : "spark",
"statn"       : "statistics",
"stats"       : "statistics",
"sugstd"      : "suggested",
"suprt"       : "support",
"swtchoff"    : "switched off",
"swth"        : "switch",
"swthd"       : "switch",
"swtych"      : "switch",

"twn"         : "town",
"tmrw"        : 'tomorrow',
"timg"        : "timing",
"tym"         : "time",
"tenserflow"  : "tensorflow",
"tf"          : "tensorflow",
"tim"         : "time",
"tmng"        : "timing",
"tmrws"       : "tomorrow",
"tody"        : "today",
"trang"       : "training",
"trning"      : "training",
"tues"        : "tuesday",

"wkend"       : "weekend",
"wk"          : "week",
"wednsdy"     : 'wednesday',
"weeknd"      : "weekend",
"wil"         : "will",
"wanna"       : "want to",
"wan"         : "want to",
"wana"        : "want to",
"want"        : "want to",
"wantd"       : "wanted",
"webianr"     : "webinar",
"wed"         : "wednesday",
"wen"         : "wednesday",
"wends"       : "wednesday",
"weekendeveng": "weekend evening",
"wek"         : "week",
"wk"          : "week",
"wekends"     : "weekends",
"wekknd"      : "weekend",
"weknd"       : "weekend",
"whn"         : "when",
"wid"         : "with",
"wii"         : "will",
"wkend"       : "weekend",
"wks"         : "week",
"wll"         : "will",
"wrg"         : "wrong",
"wrk"         : "work",
"wrng"        : "wrong",

"u"           : "you",
"ur"          : "you are",

"xlr"         : "Excelr",
"xcler"       : "Excelr",

"yo"          : "you",
"yr"          : "your",
"ystday"      : "yesterday",     

}

tech = ['ios','iot','itil','java','mba',"mongo","ms",'msc','oracle','phd','pmp','qa','qc',"rprograming",'python','tableau','sas',"spark",'sql','tensorflow','excelr']


In [37]:
gfg = TextBlob('sesion')
gfg.correct()

TextBlob("lesion")

In [ ]:
df = pd.DataFrame(leads['cleaned_text'])

unique = set(df['cleaned_text'].str.split(' ').sum())

print(list(sorted(unique)))

['', 'a', 'abdul', 'able', 'about', 'abroad', 'abt', 'accepting', 'accordingly', 'actually', 'ad', 'addres', 'address', 'admin', 'admitted', 'adress', 'adrs', 'advanced', 'after', 'afternoon', 'afterwardds', 'afterwards', 'aftr', 'aftrnun', 'again', 'agencies', 'agenda', 'agendas', 'agust', 'aim', 'aksed', 'ald', 'all', 'alomost', 'alrady', 'alrdy', 'already', 'alredy', 'also', 'altdy', 'am', 'amar', 'amarrnr', 'ambatch', 'amerpet', 'ammerpet', 'amt', 'an', 'analysis', 'analyst', 'analytics', 'and', 'anisble', 'another', 'anothr', 'anthr', 'any', 'apache', 'approach', 'approval', 'appt', 'aproving', 'are', 'area', 'arount', 'arrange', 'arrived', 'as', 'asked', 'asking', 'askng', 'asq', 'assistance', 'at', 'ateend', 'atend', 'atended', 'atleast', 'atnd', 'atnded', 'atteded', 'attedn', 'attednded', 'attend', 'attende', 'attended', 'attendind', 'attnd', 'attnde', 'attnded', 'attndng', 'attned', 'attnede', 'attneded', 'attrended', 'atttend', 'audible', 'audit', 'aug', 'august', 'augustsept

In [111]:
gfg = TextBlob('ystday')
gfg.correct()

TextBlob("stay")

In [106]:
segment("willl")

['will', 'l']

In [11]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 8.2 MB/s 


In [13]:
!pip install wordsegment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 7.6 MB/s 


In [14]:
from wordsegment import load, segment

In [15]:
load()
segment("detailsmorning")

['details', 'morning']

In [93]:
word = ['ateend', 'atend', 'atended', 'atleast', 'atnd', 'atnded', 'atteded', 'attedn', 'attednded', 'attend', 'attende', 'attended', 'attendind', 'attnd', 'attnde', 'attnded', 'attndng', 'attned', 'attnede', 'attneded', 'attrended', 'atttend']

In [17]:
from textblob import TextBlob

TextBlob("confirm")

In [95]:
leads['cleaned_text'][10]

'  rnr   share me tableau recorded session check n revert   not reachable   rnr   have rotational shifts will check details n confirm   rnr   i will confirm by tmrw   rnr   share me details of webinar on '

In [ ]:
text = '14/8/prema:ff'
text.replace(':'," ")

'14/8/prema ff'

''

In [ ]:
leads['cleaned_text'] = leads['Status information'].apply(lambda x: data_cleaning(x))

In [ ]:
leads.head(5)

,Lead Name,Location,Status,Status information,cleaned_text
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn...",share me details available in evng postpon...
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema...",cal me tmrw shared details to email share ...
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr,rnr
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...,i want only server cal busy reg for server
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ...",rnr gave info he said he will revert in ...


In [ ]:
leads['Status information'][180]

"26/7/prema: rnr 26/7/prema: shared details to email 27/7/prema: i'm looking for other options too, i will let u know 1/8/prema: droped the plans as of now"